In [26]:
import pandas as pd
import matplotlib as plt
import tensorflow as tf

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

train34_df = pd.read_csv('full_hdb_perSqm_train_f34.csv').drop(['Unnamed: 0'], axis = 1)
test34_df = pd.read_csv('full_hdb_perSqm_test_f34.csv').drop(['Unnamed: 0'], axis = 1)
train16_df = pd.read_csv('full_hdb_perSqm_train_f16.csv').drop(['Unnamed: 0'], axis = 1)
test16_df = pd.read_csv('full_hdb_perSqm_test_f16.csv').drop(['Unnamed: 0'], axis = 1)

train34_df


,lat,minPrimary_transitTime,min_dis,remaining_lease,DBSS,Improved,Model A,New Generation,Type S1,Type S2,...,BUKIT MERAH,CENTRAL AREA,CHOA CHU KANG,CLEMENTI,JURONG WEST,KALLANG/WHAMPOA,QUEENSTOWN,WOODLANDS,YISHUN,resale_price_per_sqm
0,1.377567,759.0,0.001040,93.166667,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,6410.714286
1,1.371036,368.0,0.018341,60.833333,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,5186.813187
2,1.430421,964.0,0.005845,71.083333,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,5335.365854
3,1.352865,448.0,0.009913,94.833333,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,6691.176471
4,1.371233,454.0,0.005618,61.000000,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,5476.190476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13806,1.341138,514.0,0.003085,48.833333,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,4908.045977
13807,1.329478,868.0,0.006544,57.083333,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,5176.470588
13808,1.389799,254.0,0.002423,91.833333,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,6246.107527
13809,1.338132,397.0,0.004211,76.416667,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,5709.090909


In [27]:
test34_df


,lat,minPrimary_transitTime,min_dis,remaining_lease,DBSS,Improved,Model A,New Generation,Type S1,Type S2,...,BUKIT MERAH,CENTRAL AREA,CHOA CHU KANG,CLEMENTI,JURONG WEST,KALLANG/WHAMPOA,QUEENSTOWN,WOODLANDS,YISHUN,resale_price_per_sqm
0,1.367908,871.0,0.002827,53.500000,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,6122.448980
1,1.374001,612.0,0.011506,62.083333,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,6440.000000
2,1.364612,834.0,0.007193,88.750000,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,9071.428571
3,1.365588,1052.0,0.010096,53.583333,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,5671.641791
4,1.365698,1347.0,0.010306,56.083333,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,4775.280899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5914,1.422602,879.0,0.011602,87.833333,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,5855.670103
5915,1.419920,608.0,0.012480,88.916667,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,5978.494624
5916,1.427238,865.0,0.002624,59.583333,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,5879.120879
5917,1.414291,934.0,0.007527,93.583333,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,5619.469027


In [28]:
train16_df


,lat,remaining_lease,DBSS,Model A,New Generation,Type S1,01 TO 03,22 TO 24,25 TO 27,28 TO 30,34 TO 36,BUKIT MERAH,CENTRAL AREA,QUEENSTOWN,WOODLANDS,resale_price_per_sqm
0,1.377567,93.166667,False,False,False,False,False,False,False,False,False,False,False,False,False,6410.714286
1,1.371036,60.833333,False,False,True,False,True,False,False,False,False,False,False,False,False,5186.813187
2,1.430421,71.083333,False,False,False,False,False,False,False,False,False,False,False,False,True,5335.365854
3,1.352865,94.833333,False,True,False,False,False,False,False,False,False,False,False,False,False,6691.176471
4,1.371233,61.000000,False,False,False,False,False,False,False,False,False,False,False,False,False,5476.190476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13806,1.341138,48.833333,False,False,False,False,True,False,False,False,False,False,False,False,False,4908.045977
13807,1.329478,57.083333,False,False,True,False,False,False,False,False,False,False,False,False,False,5176.470588
13808,1.389799,91.833333,False,True,False,False,False,False,False,False,False,False,False,False,False,6246.107527
13809,1.338132,76.416667,False,False,False,False,False,False,False,False,False,False,False,False,False,5709.090909


In [29]:
test16_df

,lat,remaining_lease,DBSS,Model A,New Generation,Type S1,01 TO 03,22 TO 24,25 TO 27,28 TO 30,34 TO 36,BUKIT MERAH,CENTRAL AREA,QUEENSTOWN,WOODLANDS,resale_price_per_sqm
0,1.367908,53.500000,False,False,False,False,False,False,False,False,False,False,False,False,False,6122.448980
1,1.374001,62.083333,False,False,False,False,True,False,False,False,False,False,False,False,False,6440.000000
2,1.364612,88.750000,False,True,False,False,False,False,True,False,False,False,False,False,False,9071.428571
3,1.365588,53.583333,False,False,True,False,False,False,False,False,False,False,False,False,False,5671.641791
4,1.365698,56.083333,False,False,True,False,False,False,False,False,False,False,False,False,False,4775.280899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5914,1.422602,87.833333,False,True,False,False,False,False,False,False,False,False,False,False,False,5855.670103
5915,1.419920,88.916667,False,True,False,False,False,False,False,False,False,False,False,False,False,5978.494624
5916,1.427238,59.583333,False,False,True,False,False,False,False,False,False,False,False,False,False,5879.120879
5917,1.414291,93.583333,False,False,False,False,True,False,False,False,False,False,False,False,False,5619.469027


In [30]:
para_dict = {
    'hidden_units' : [7, 8, 9],
    'batch_size' : [16, 32, 64],
    'learning_rate' : [[0.008, 0.01], [0.0008, 0.001]]
}

def create_model(columns, units = 8, activation = 'relu', loss = 'mae', optimizer = 'SGD', metrics = 'mae'):
    
    model = Sequential([
        Dense(units, input_shape = (len(columns), ), activation = activation),
        Dense(1, activation = activation)
    ])

    model.compile(
        optimizer = optimizer,
        loss = loss,
        metrics = metrics
    )

    return model


In [31]:
'''
Model 34
'''

# train set
y_train = train34_df['resale_price_per_sqm']
x_train = train34_df.drop('resale_price_per_sqm', axis = 1)

# test set
y_test = test34_df[['resale_price_per_sqm']]
x_test = test34_df.drop('resale_price_per_sqm', axis = 1)

x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5919 entries, 0 to 5918
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lat                     5919 non-null   float64
 1   minPrimary_transitTime  5919 non-null   float64
 2   min_dis                 5919 non-null   float64
 3   remaining_lease         5919 non-null   float64
 4   DBSS                    5919 non-null   bool   
 5   Improved                5919 non-null   bool   
 6   Model A                 5919 non-null   bool   
 7   New Generation          5919 non-null   bool   
 8   Type S1                 5919 non-null   bool   
 9   Type S2                 5919 non-null   bool   
 10  2 ROOM                  5919 non-null   bool   
 11  01 TO 03                5919 non-null   bool   
 12  04 TO 06                5919 non-null   bool   
 13  16 TO 18                5919 non-null   bool   
 14  19 TO 21                5919 non-null   

In [32]:
scaler = StandardScaler()

x_scaled_train = scaler.fit_transform(x_train)

x_scaled_train = pd.DataFrame(x_scaled_train, columns = x_train.columns)

x_scaled_test = scaler.fit_transform(x_test)

x_scaled_test = pd.DataFrame(x_scaled_test, columns = x_test.columns)

x_scaled_train

,lat,minPrimary_transitTime,min_dis,remaining_lease,DBSS,Improved,Model A,New Generation,Type S1,Type S2,...,BISHAN,BUKIT MERAH,CENTRAL AREA,CHOA CHU KANG,CLEMENTI,JURONG WEST,KALLANG/WHAMPOA,QUEENSTOWN,WOODLANDS,YISHUN
0,0.156048,-0.026266,-1.299018,1.261190,-0.113286,1.800836,-0.816201,-0.360699,-0.039943,-0.022519,...,-0.121219,-0.198981,-0.082337,4.855382,-0.145157,-0.253995,-0.18119,-0.159825,-0.305932,-0.278484
1,0.007990,-1.229159,3.027768,-0.878525,-0.113286,-0.555298,-0.816201,2.772393,-0.039943,-0.022519,...,-0.121219,-0.198981,-0.082337,-0.205957,-0.145157,-0.253995,-0.18119,-0.159825,-0.305932,-0.278484
2,1.354411,0.604407,-0.097186,-0.200213,-0.113286,-0.555298,-0.816201,-0.360699,-0.039943,-0.022519,...,-0.121219,-0.198981,-0.082337,-0.205957,-0.145157,-0.253995,-0.18119,-0.159825,3.268705,-0.278484
3,-0.404001,-0.983043,0.920124,1.371485,-0.113286,-0.555298,1.225188,-0.360699,-0.039943,-0.022519,...,-0.121219,-0.198981,-0.082337,-0.205957,-0.145157,-0.253995,-0.18119,-0.159825,-0.305932,-0.278484
4,0.012449,-0.964584,-0.154118,-0.867495,-0.113286,-0.555298,-0.816201,-0.360699,-0.039943,-0.022519,...,-0.121219,-0.198981,-0.082337,-0.205957,-0.145157,-0.253995,-0.18119,-0.159825,-0.305932,-0.278484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13806,-0.669887,-0.779997,-0.787476,-1.672645,-0.113286,1.800836,-0.816201,-0.360699,-0.039943,-0.022519,...,-0.121219,-0.198981,-0.082337,-0.205957,-0.145157,-0.253995,-0.18119,-0.159825,-0.305932,-0.278484
13807,-0.934243,0.309068,0.077641,-1.126687,-0.113286,-0.555298,-0.816201,2.772393,-0.039943,-0.022519,...,-0.121219,-0.198981,-0.082337,-0.205957,-0.145157,-0.253995,-0.18119,-0.159825,-0.305932,-0.278484
13808,0.433403,-1.579874,-0.953137,1.172955,-0.113286,-0.555298,1.225188,-0.360699,-0.039943,-0.022519,...,-0.121219,-0.198981,-0.082337,-0.205957,-0.145157,-0.253995,-0.18119,-0.159825,-0.305932,-0.278484
13809,-0.738033,-1.139942,-0.505835,0.152730,-0.113286,1.800836,-0.816201,-0.360699,-0.039943,-0.022519,...,-0.121219,-0.198981,-0.082337,-0.205957,-0.145157,3.937080,-0.18119,-0.159825,-0.305932,-0.278484


In [33]:
# 用梯度下降 - 小批量随机梯度下降（MBGD）- mae + sgd

result34_dict = {
    'units': [],
    'batch_size': [],
    'learning_rate': [],
    'minimum_mae_error': []
}

for units in para_dict['hidden_units']:
    for b_size in para_dict['batch_size']:
        for rate in para_dict['learning_rate'][0]:

            early_stopping = EarlyStopping(monitor = 'val_loss',
                               patience = 10,
                               restore_best_weights = True)

            optimizer = SGD(learning_rate = rate)

            model = create_model(columns = x_train.columns, units = units, optimizer = optimizer)

            best_model = model.fit(
                x_scaled_train, y_train,
                validation_data = (x_scaled_test, y_test),
                batch_size = b_size,
                epochs = 250,
                callbacks = [early_stopping]
            )

            min_mae = early_stopping.best

            result34_dict['units'].append(units)
            result34_dict['batch_size'].append(b_size)
            result34_dict['learning_rate'].append(rate)
            result34_dict['minimum_mae_error'].append(min_mae)



Epoch 1/250


864/864 [==============================] - 2s 2ms/step - loss: 4385.7329 - mae: 4385.7329 - val_loss: 535.1006 - val_mae: 535.1006
Epoch 2/250
864/864 [==============================] - 2s 2ms/step - loss: 522.1991 - mae: 522.1991 - val_loss: 553.0164 - val_mae: 553.0164
Epoch 3/250
864/864 [==============================] - 2s 2ms/step - loss: 524.3036 - mae: 524.3036 - val_loss: 532.6707 - val_mae: 532.6707
Epoch 4/250
864/864 [==============================] - 2s 2ms/step - loss: 512.0093 - mae: 512.0093 - val_loss: 526.1748 - val_mae: 526.1748
Epoch 5/250
864/864 [==============================] - 2s 2ms/step - loss: 503.6854 - mae: 503.6854 - val_loss: 491.7916 - val_mae: 491.7916
Epoch 6/250
864/864 [==============================] - 2s 2ms/step - loss: 482.8032 - mae: 482.8032 - val_loss: 481.4672 - val_mae: 481.4672
Epoch 7/250
864/864 [==============================] - 2s 2ms/step - loss: 460.5576 - mae: 460.5576 - val_loss: 479.0305 - val_mae: 479.0305
Epoch 8/250
864/864 [==

In [34]:
result34_sgd_df = pd.DataFrame(result34_dict)

result34_sgd_df.to_csv('result34_sgd.csv')

result34_sgd_df

,units,batch_size,learning_rate,minimum_mae_error
0,7,16,0.008,419.546722
1,7,16,0.010,404.253998
2,7,32,0.008,416.352631
3,7,32,0.010,400.240417
4,7,64,0.008,404.492615
5,7,64,0.010,411.879272
6,8,16,0.008,405.715485
7,8,16,0.010,420.990295
8,8,32,0.008,400.517303
9,8,32,0.010,404.473358


In [35]:
# 用梯度下降 - 小批量随机梯度下降（MBGD）- mse + adam

result34_adam_dict = {
    'units': [],
    'batch_size': [],
    'learning_rate': [],
    'minimum_mae_error': []
}

for units in para_dict['hidden_units']:
    for b_size in para_dict['batch_size']:
        for rate in para_dict['learning_rate'][0]:

            early_stopping = EarlyStopping(monitor = 'val_mae',
                               patience = 10,
                               restore_best_weights = True)

            optimizer = Adam(learning_rate = rate)

            model = create_model(columns = x_train.columns, units = units, loss = 'mse', optimizer = optimizer)

            best_model = model.fit(
                x_scaled_train, y_train,
                validation_data = (x_scaled_test, y_test),
                batch_size = b_size,
                epochs = 250,
                callbacks = [early_stopping]
            )

            min_mae = early_stopping.best

            result34_adam_dict['units'].append(units)
            result34_adam_dict['batch_size'].append(b_size)
            result34_adam_dict['learning_rate'].append(rate)
            result34_adam_dict['minimum_mae_error'].append(min_mae)

Epoch 1/250
864/864 [==============================] - 2s 2ms/step - loss: 32984368.0000 - mae: 5573.8271 - val_loss: 22238200.0000 - val_mae: 4583.3091
Epoch 2/250
864/864 [==============================] - 2s 2ms/step - loss: 11565702.0000 - mae: 3122.6870 - val_loss: 4104294.7500 - val_mae: 1752.9337
Epoch 3/250
864/864 [==============================] - 2s 2ms/step - loss: 1949318.6250 - mae: 1068.1791 - val_loss: 1073955.3750 - val_mae: 771.2946
Epoch 4/250
864/864 [==============================] - 2s 2ms/step - loss: 818598.3125 - mae: 674.0406 - val_loss: 715207.3125 - val_mae: 632.5662
Epoch 5/250
864/864 [==============================] - 2s 2ms/step - loss: 610014.0000 - mae: 577.7812 - val_loss: 595390.6250 - val_mae: 569.1544
Epoch 6/250
864/864 [==============================] - 2s 2ms/step - loss: 536474.7500 - mae: 537.9304 - val_loss: 545388.8750 - val_mae: 539.3376
Epoch 7/250
864/864 [==============================] - 2s 2ms/step - loss: 499802.7500 - mae: 518.1223 -

In [36]:
result34_adam_df = pd.DataFrame(result34_adam_dict)

result34_adam_df.to_csv('result34_adam.csv')

result34_adam_df

,units,batch_size,learning_rate,minimum_mae_error
0,7,16,0.008,438.642181
1,7,16,0.010,421.768707
2,7,32,0.008,444.606171
3,7,32,0.010,427.654205
4,7,64,0.008,431.937073
5,7,64,0.010,436.990997
6,8,16,0.008,430.238525
7,8,16,0.010,427.203339
8,8,32,0.008,424.160950
9,8,32,0.010,431.932556


In [37]:
'''
Model 16
'''

# train set
y_train = train16_df['resale_price_per_sqm']
x_train = train16_df.drop('resale_price_per_sqm', axis = 1)

# test set
y_test = test16_df[['resale_price_per_sqm']]
x_test = test16_df.drop('resale_price_per_sqm', axis = 1)

x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5919 entries, 0 to 5918
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   lat              5919 non-null   float64
 1   remaining_lease  5919 non-null   float64
 2   DBSS             5919 non-null   bool   
 3   Model A          5919 non-null   bool   
 4   New Generation   5919 non-null   bool   
 5   Type S1          5919 non-null   bool   
 6   01 TO 03         5919 non-null   bool   
 7   22 TO 24         5919 non-null   bool   
 8   25 TO 27         5919 non-null   bool   
 9   28 TO 30         5919 non-null   bool   
 10  34 TO 36         5919 non-null   bool   
 11  BUKIT MERAH      5919 non-null   bool   
 12  CENTRAL AREA     5919 non-null   bool   
 13  QUEENSTOWN       5919 non-null   bool   
 14  WOODLANDS        5919 non-null   bool   
dtypes: bool(13), float64(2)
memory usage: 167.8 KB


In [38]:
scaler = StandardScaler()

x_scaled_train = scaler.fit_transform(x_train)

x_scaled_train = pd.DataFrame(x_scaled_train, columns = x_train.columns)

x_scaled_test = scaler.fit_transform(x_test)

x_scaled_test = pd.DataFrame(x_scaled_test, columns = x_test.columns)

x_scaled_train

,lat,remaining_lease,DBSS,Model A,New Generation,Type S1,01 TO 03,22 TO 24,25 TO 27,28 TO 30,34 TO 36,BUKIT MERAH,CENTRAL AREA,QUEENSTOWN,WOODLANDS
0,0.156048,1.261190,-0.113286,-0.816201,-0.360699,-0.039943,-0.450726,-0.115237,-0.100095,-0.076808,-0.057808,-0.198981,-0.082337,-0.159825,-0.305932
1,0.007990,-0.878525,-0.113286,-0.816201,2.772393,-0.039943,2.218645,-0.115237,-0.100095,-0.076808,-0.057808,-0.198981,-0.082337,-0.159825,-0.305932
2,1.354411,-0.200213,-0.113286,-0.816201,-0.360699,-0.039943,-0.450726,-0.115237,-0.100095,-0.076808,-0.057808,-0.198981,-0.082337,-0.159825,3.268705
3,-0.404001,1.371485,-0.113286,1.225188,-0.360699,-0.039943,-0.450726,-0.115237,-0.100095,-0.076808,-0.057808,-0.198981,-0.082337,-0.159825,-0.305932
4,0.012449,-0.867495,-0.113286,-0.816201,-0.360699,-0.039943,-0.450726,-0.115237,-0.100095,-0.076808,-0.057808,-0.198981,-0.082337,-0.159825,-0.305932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13806,-0.669887,-1.672645,-0.113286,-0.816201,-0.360699,-0.039943,2.218645,-0.115237,-0.100095,-0.076808,-0.057808,-0.198981,-0.082337,-0.159825,-0.305932
13807,-0.934243,-1.126687,-0.113286,-0.816201,2.772393,-0.039943,-0.450726,-0.115237,-0.100095,-0.076808,-0.057808,-0.198981,-0.082337,-0.159825,-0.305932
13808,0.433403,1.172955,-0.113286,1.225188,-0.360699,-0.039943,-0.450726,-0.115237,-0.100095,-0.076808,-0.057808,-0.198981,-0.082337,-0.159825,-0.305932
13809,-0.738033,0.152730,-0.113286,-0.816201,-0.360699,-0.039943,-0.450726,-0.115237,-0.100095,-0.076808,-0.057808,-0.198981,-0.082337,-0.159825,-0.305932


In [39]:
# 用梯度下降 - 小批量随机梯度下降（MBGD）- mae + sgd

result16_dict = {
    'units': [],
    'batch_size': [],
    'learning_rate': [],
    'minimum_mae_error': []
}

for units in para_dict['hidden_units']:
    for b_size in para_dict['batch_size']:
        for rate in para_dict['learning_rate'][0]:

            early_stopping = EarlyStopping(monitor = 'val_loss',
                               patience = 10,
                               restore_best_weights = True)

            optimizer = SGD(learning_rate = rate)

            model = create_model(columns = x_train.columns, units = units, optimizer = optimizer)

            best_model = model.fit(
                x_scaled_train, y_train,
                validation_data = (x_scaled_test, y_test),
                batch_size = b_size,
                epochs = 250,
                callbacks = [early_stopping]
            )

            min_mae = early_stopping.best

            result16_dict['units'].append(units)
            result16_dict['batch_size'].append(b_size)
            result16_dict['learning_rate'].append(rate)
            result16_dict['minimum_mae_error'].append(min_mae)


Epoch 1/250
864/864 [==============================] - 2s 2ms/step - loss: 4333.5039 - mae: 4333.5039 - val_loss: 640.2736 - val_mae: 640.2736
Epoch 2/250
864/864 [==============================] - 2s 2ms/step - loss: 624.3925 - mae: 624.3925 - val_loss: 654.2614 - val_mae: 654.2614
Epoch 3/250
864/864 [==============================] - 2s 2ms/step - loss: 621.2056 - mae: 621.2056 - val_loss: 634.6089 - val_mae: 634.6089
Epoch 4/250
864/864 [==============================] - 2s 2ms/step - loss: 619.6724 - mae: 619.6724 - val_loss: 631.2358 - val_mae: 631.2358
Epoch 5/250
864/864 [==============================] - 2s 2ms/step - loss: 615.4807 - mae: 615.4807 - val_loss: 624.4448 - val_mae: 624.4448
Epoch 6/250
864/864 [==============================] - 2s 2ms/step - loss: 610.7493 - mae: 610.7493 - val_loss: 618.1853 - val_mae: 618.1853
Epoch 7/250
864/864 [==============================] - 2s 2ms/step - loss: 599.5564 - mae: 599.5564 - val_loss: 600.0408 - val_mae: 600.0408
Epoch 8/250

In [40]:
result16_sgd_df = pd.DataFrame(result16_dict)

result16_sgd_df.to_csv('result16_sgd.csv')

result16_sgd_df

,units,batch_size,learning_rate,minimum_mae_error
0,7,16,0.008,533.118286
1,7,16,0.010,544.053589
2,7,32,0.008,524.502625
3,7,32,0.010,524.062134
4,7,64,0.008,6046.202637
5,7,64,0.010,548.358154
6,8,16,0.008,528.784241
7,8,16,0.010,520.338440
8,8,32,0.008,525.308899
9,8,32,0.010,542.368774


In [41]:
# 用梯度下降 - 小批量随机梯度下降（MBGD）- mse + adam

result16_adam_dict = {
    'units': [],
    'batch_size': [],
    'learning_rate': [],
    'minimum_mae_error': []
}

for units in para_dict['hidden_units']:
    for b_size in para_dict['batch_size']:
        for rate in para_dict['learning_rate'][0]:

            early_stopping = EarlyStopping(monitor = 'val_mae',
                               patience = 10,
                               restore_best_weights = True)

            optimizer = Adam(learning_rate = rate)

            model = create_model(columns = x_train.columns, units = units, loss = 'mse', optimizer = optimizer)

            best_model = model.fit(
                x_scaled_train, y_train,
                validation_data = (x_scaled_test, y_test),
                batch_size = b_size,
                epochs = 250,
                callbacks = [early_stopping]
            )

            min_mae = early_stopping.best

            result16_adam_dict['units'].append(units)
            result16_adam_dict['batch_size'].append(b_size)
            result16_adam_dict['learning_rate'].append(rate)
            result16_adam_dict['minimum_mae_error'].append(min_mae)

Epoch 1/250
864/864 [==============================] - 2s 2ms/step - loss: 33764916.0000 - mae: 5639.6899 - val_loss: 24582466.0000 - val_mae: 4798.4893
Epoch 2/250
864/864 [==============================] - 2s 2ms/step - loss: 14529671.0000 - mae: 3514.8708 - val_loss: 6433597.5000 - val_mae: 2211.1931
Epoch 3/250
864/864 [==============================] - 2s 2ms/step - loss: 3394161.7500 - mae: 1351.4941 - val_loss: 1792912.2500 - val_mae: 867.7614
Epoch 4/250
864/864 [==============================] - 2s 2ms/step - loss: 1539530.8750 - mae: 787.2482 - val_loss: 1318663.0000 - val_mae: 732.8066
Epoch 5/250
864/864 [==============================] - 2s 2ms/step - loss: 1256597.3750 - mae: 728.3976 - val_loss: 1144379.6250 - val_mae: 703.9740
Epoch 6/250
864/864 [==============================] - 2s 2ms/step - loss: 1103670.3750 - mae: 699.6934 - val_loss: 1028257.7500 - val_mae: 685.5306
Epoch 7/250
864/864 [==============================] - 2s 2ms/step - loss: 998530.8125 - mae: 680.

In [42]:
result16_adam_df = pd.DataFrame(result16_adam_dict)

result16_adam_df.to_csv('result16_adam.csv')

result16_adam_df

,units,batch_size,learning_rate,minimum_mae_error
0,7,16,0.008,551.521484
1,7,16,0.010,552.343750
2,7,32,0.008,550.431946
3,7,32,0.010,543.401855
4,7,64,0.008,567.597595
5,7,64,0.010,570.497620
6,8,16,0.008,560.813599
7,8,16,0.010,6046.201172
8,8,32,0.008,568.569031
9,8,32,0.010,561.833313
